# Adaptive PDE discretizations on cartesian grids
## Volume : Algorithmic tools
## Part : Tensor decomposition techniques
## Chapter : Voronoi's reduction, in dimension 6, application to Hooke tensor decomposition

We rely on Voronoi's first reduction of quadratic forms to decompose symmetric positive definite matrices in dimension $\leq 6$, in a form which resembles the eigenvalue-eigenvector decomposition but with integer offsets:
$$
    D = \sum_{1 \leq i \leq d'} \rho_i e_i e_i^T
$$
where $d'=d(d+1)/2$ (except $d'=12$ in dimension $4$ in our implementation), $\rho_i\geq 0$, and $e_i \in Z^d$.

The six-dimensional case is especially interesting for its applications to the Hooke elasticity tensor in three dimensional elasticity. The approach is (very) unlikely to extend to higher dimensions (including dimension 7), in a computationally efficient manner, due to a combinatorial explosion in Voronoi's theory.

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"../..") # Allow import of agd from parent directory (useless if conda package installed)
#from Miscellaneous import TocTools; TocTools.displayTOC('TensorVoronoi','Algo')

In [2]:
from agd import LinearParallel as lp
from agd.Selling import GatherByOffset
from agd import AutomaticDifferentiation as ad
from agd.Metrics import Seismic
from agd.Plotting import savefig; #savefig.dirName = 'Figures/TensorVoronoi'

The routines for tensor decomposition are for efficiency purposes provided in a small c++ library, named FileVDQ where VDQ stands for "Voronoi Decomposition of Quadratic forms". This is in contrast with the two and three dimensional cases, where the decomposition algorithm is coded in Python (the c++ library can also be used in smaller dimensions). A function named `VoronoiDecomposition` provides the interface.

In [3]:
from agd.Eikonal import VoronoiDecomposition

In [4]:
import numpy as np; allclose = np.allclose
import matplotlib.pyplot as plt
import time

### 0.1 Optional configuration

Uncomment the following line to use the GPU implementation of Voronoi's decomposition.

In [5]:
VoronoiDecomposition.default_mode = 'gpu_transfer'; allclose = ad.cupy_friendly(allclose)

Setting float32 compatible default values atol=rtol=1e-5 in np.allclose


Choose to use, or not, large instances by uncommenting the following line (computation time may become longer).

In [6]:
large_instances = False

## 1. Computing the decomposition of a tensor

We illustrate our tensor decomposition method on random positive definite matrices, of the form 
$$
    D = A^T A,
$$
where $A$ is a square matrix with random coefficients w.r.t. the Gaussian normal law.

In [7]:
def MakeRandomTensor(dim, shape=tuple(), relax=0.01):
    A = np.random.standard_normal( (dim,dim) + shape )
    identity = np.eye(dim).reshape((6,6)+(1,)*len(shape))
    return lp.dot_AA(lp.transpose(A),A) +relax*identity

In [8]:
# For reproducibility, we fix the random seed
np.random.seed(42) 

The inserse operation to tensor decomposition is, of course, reconstruction, defined by 
$$
    (\lambda_i, e_i)_{i=1}^I \mapsto D = \sum_{1 \leq i \leq I} \lambda_i e_i e_i^T
$$

In [9]:
def Reconstruct(coefs,offsets):
     return lp.mult(coefs,lp.outer_self(offsets)).sum(2)

In [10]:
def LInfNorm(a):
    return np.max(np.abs(a))

### 1.1 Case of a $6\times 6$ tensor

In [13]:
np.random.seed(42) # Reproducibility
D = MakeRandomTensor(6)

The compile time can be quite long on GPU. The execution time may not be instantaneous either, especially on GPU.

In [14]:
coefs,offsets = VoronoiDecomposition(D)

Our decomposition of a $6\times 6$ symmetric positive definite matrix involves $21$ (non-negative) weights and (integral) offsets.

In [15]:
print("Coefficients : ", coefs)
print("Offsets : \n", offsets.astype(int))

Coefficients :  [0.02547066 0.02839559 0.08069724 0.09730089 0.11464643 0.14023937
 0.21295398 0.27030981 0.28218755 0.33612779 0.42452446 0.47649354
 0.5327549  0.54081041 0.57074392 0.71876055 0.73967278 0.74280208
 0.77983648 0.92753559 0.99830532]
Offsets : 
 [[ 1  1  0  0  1  0  1  1  1  0  0 -1 -1  0  1  0  0  0  1  0  1]
 [-1  0  1 -1  0 -1  2  0  1  0 -1  0 -1  1  1 -1  0  1  0 -2 -1]
 [ 0 -1 -1 -1  0  1 -1 -2  0  1  1  0  1  0  0  1  1  0  1 -1 -1]
 [ 1  1 -1  1  0  1  0  0  1  1  0  0  0  0  0  0  1  1  1  0  1]
 [-1 -1  0 -1  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0 -1 -1]
 [-1  0 -1  0  0  0  0  0  0 -1 -1 -1 -1 -1 -1  0  0  0  0  0  1]]


By design, the coefficients are non-negative, and the reconstruction is exact up to numerical precision.

In [16]:
print("Minimal coefficient : ", np.min(coefs))
print("Reconstruction error : ", LInfNorm(D-Reconstruct(coefs,offsets)))
assert np.allclose(D,Reconstruct(coefs,offsets))

Minimal coefficient :  0.025470662862062454
Reconstruction error :  3.241269668663449e-06


### 1.2 Case of a field of tensors

On the CPU, the decomposition time increases linearly with the number of tensors.
On the GPU, the decomposition runs on a single thread, and there e.g. $2560$ of them on a nvidia $1080$ class card. 
Therefore time is rather insensitive to the number of tensors until that number is reached.

In [17]:
def decomp_time(n):
    np.random.seed(42)
    D = MakeRandomTensor(6,(n,))
    start = time.time()
    coefs,offsets = VoronoiDecomposition(D)
    print(f"Decomposition of {n} matrices completed in {time.time()-start} seconds")
    print("Tensor shape: ",D.shape,", max reconstruction error : ",np.max(np.abs(D-Reconstruct(coefs,offsets))))
#    assert allclose(D,Reconstruct(coefs,offsets))

In [18]:
decomp_time(10)

Decomposition of 10 matrices completed in 6.59154486656189 seconds
Tensor shape:  (6, 6, 10) , max reconstruction error :  1.6817056083251458e-05


In [63]:
decomp_time(100)

Decomposition of 100 matrices completed in 11.787024974822998 seconds
Tensor shape:  (6, 6, 100) , max reconstruction error :  6.10084326950755e-05


In [64]:
decomp_time(1000)

Decomposition of 1000 matrices completed in 18.521484851837158 seconds
Tensor shape:  (6, 6, 1000) , max reconstruction error :  0.00040835346870338185


On the GPU, decomposition time should be mostly insensitive to $n\leq 2500$, and scale linearly with $n$ beyond that limit. 

<!---
TODO : investigate this failure. CPU also ? 
Decomposition of 10000 matrices completed in 73.4375216960907 seconds
Tensor shape:  (6, 6, 10000) , max reconstruction error :  7.562082901426454
--->

In [65]:
if VoronoiDecomposition.default_mode = 'gpu_transfer' and large_instances: decomp_time(10000)

Decomposition of 10000 matrices completed in 73.4375216960907 seconds
Tensor shape:  (6, 6, 10000) , max reconstruction error :  7.562082901426454


### 1.3 Case of a Hooke tensor

The hooke tensor is a $(3,3,3,3)$ shaped tensor characterizing the elastic properties of a physical medium.
It has various symmetries, and can be regarded as a quadratic form on the space of $6 \times 6$ symmetric matrices.

In [11]:
hooke,ρ = Seismic.Hooke.mica # Hooke tensor of the mica medium, vertically aligned
hooke = hooke.rotate_by(0.5, (1,2,3)) # Rotate along some arbitrary axis

In [13]:
coefs,moffsets = hooke.Selling()

The quadratic form define by the Hooke tensor is reformulated, using Voronoi's decomposition, in the form
$$
    \sum_{1 \leq i \leq d'} \rho_i \mathrm{Tr}(M D_i)^2
$$
where $d'=21$, $\rho_i\geq 0$, $D_i$ is a $3 \times 3$ symmetric matrix with integer entries.

In [ ]:
np.random.seed(42)
m = MakeRandomTensor(3)


### 1.1 Case of a  $6 \times 6$ tensor

In [16]:
#nsupport = [21, 30, 36, 21, 27, 22, 21]

In [15]:
#coefs,offsets = VoronoiDecomposition(D6)

In [16]:
assert False
for i in range(300):
    np.random.seed(i)
    D = MakeRandomTensor(6)
    _,_,vertex,_ = VoronoiDecomposition(D,single_step=False)
    if vertex!=2:
        print(i,vertex)

AssertionError: 

286,(268,97,154),255

In [19]:
np.random.seed(255)
D = MakeRandomTensor(6)
m,a,vertex,objective = VoronoiDecomposition(D,steps="Single")
assert vertex!=2
nsupport[vertex]

27

In [20]:
VoronoiDecomposition(D,steps="Both",traits = {"nsupport_max":30})

CUDARuntimeError: cudaErrorIllegalAddress: an illegal memory access was encountered

In [ ]:
#coefs,offsets =  VoronoiDecomposition(D,traits = {"nsupport_max":max(23,nsupport[vertex])})
coefs,offsets =  VoronoiDecomposition(D,traits = {"nsupport_max":31})
print("Reconstruction error : ", LInfNorm(D-Reconstruct(coefs,offsets)))

In [ ]:
D

In [ ]:
coefs

In [ ]:
(39*17*14//2)

In [ ]:
#coefs,offsets = VoronoiDecomposition(D6)

Our decomposition, of a $4 \times 4$ SPD tensor, involves either $10$ or $12$ coefficients and offsets. 
If the tensor is randomly generated, then each possibility arises with positive probability, in approximately half the cases.

For uniformity of the data structures, we always return $12$ coefficients and offsets, but the last two are often zero.

In [ ]:
print("Coefficients : ", coefs)
print("Offsets : \n", offsets.astype(int))

By design, the coefficients are non-negative, and the reconstruction is exact up to numerical precision.

In [ ]:
print("Minimal coefficient : ", np.min(coefs))
print("Reconstruction error : ", LInfNorm(D6-Reconstruct(coefs,offsets)))
assert np.allclose(D6,Reconstruct(coefs,offsets))

2 1
21 1
23 1
39 1
40 1
43 1
75 1
82 1
83 1
97 5
99 1
114 1
122 1
131 1
135 1
145 1
154 5
162 1
184 5
186 1
198 3
201 1
205 1
217 1
218 1
235 1
237 1
255 4
261 1
268 5
273 1
279 1
284 1
286 3
298 1

## 1.2 A family of tensors

In [ ]:
def Interpolate(a,b,T=np.linspace(0,1,100)):
    return T, np.moveaxis(np.array([(1-t)*a + t*b for t in T]),0,-1)

In [ ]:
T_interp, D_interp = Interpolate(MakeRandomTensor(6),MakeRandomTensor(6))

In [ ]:
%%time
coefs,offsets = VoronoiDecomposition(D_interp)

In [ ]:
print("Reconstruction error : ", LInfNorm(D_interp - Reconstruct(coefs,offsets)))
assert np.allclose(D_interp, Reconstruct(coefs,offsets),atol=1e-5)

In [ ]:
decomp = GatherByOffset(T_interp,coefs,offsets)

In [ ]:
fig = plt.figure(figsize=(20,10))
for offset,(time,coef) in decomp.items():
    plt.plot(time,coef)
plt.legend(decomp.keys(),ncol=3)
savefig(fig,"Coefs_Vor4.pdf")

In [ ]:
fig = plt.figure(figsize=(20,10))
for offset,(time,coef) in decomp.items():
    plt.plot(time,coef)
plt.legend(decomp.keys(),ncol=3)
savefig(fig,"Coefs_Vor4.pdf")